In [104]:
import numpy as np
import pandas as pd
from PIL import Image
import os
from zipfile import ZipFile

In [105]:
ZipFile('gan-getting-started.zip', 'r').extractall(path = 'images')

In [65]:
images = pd.DataFrame()
images_list = []
dir = 'images/monet_jpg'
for image in os.listdir(dir):
    images_list.append(np.asarray(Image.open(dir +'/' + image)))
images['real_paintings'] = pd.Series(images_list)
images.head()

,real_paintings
0,"[[[180, 178, 139], [193, 191, 153], [202, 202,..."
1,"[[[15, 36, 17], [66, 87, 70], [93, 111, 97], [..."
2,"[[[178, 196, 206], [190, 208, 218], [199, 217,..."
3,"[[[160, 164, 191], [163, 169, 195], [164, 170,..."
4,"[[[154, 139, 118], [163, 148, 127], [163, 148,..."


In [66]:
len(images)

300

In [71]:
Image.fromarray(images['real_paintings'][0]).show()

In [72]:
Image.fromarray(np.rot90(images['real_paintings'][0], 3)).show()

In [73]:
#Image.fromarray(np.flip(np.flip(images[0],0), 1)).show()
#Image.fromarray(np.flip(images[0],2)).show()
Image.fromarray(np.rot90(np.flip(np.flip(images['real_paintings'][0],0), 1), 3)).show()

In [67]:
flipped_image_list = []
for image in images_list:
    flipped_image_list.append(np.flip(image, 0))
    flipped_image_list.append(np.flip(image, 1))
    flipped_image_list.append(np.flip(np.flip(image, 0), 1))
    flipped_image_list.append(np.rot90(image))
    flipped_image_list.append(np.rot90(image, 3))
    flipped_image_list.append(np.flip(np.rot90(image), 0))
    flipped_image_list.append(np.flip(np.rot90(image), 1))
    flipped_image_list.append(np.flip(image, -1))
    flipped_image_list.append(np.flip(np.flip(image, 0), -1))
    flipped_image_list.append(np.flip(np.flip(image, 1), -1))
    flipped_image_list.append(np.flip(np.flip(np.flip(image, 0), 1), -1))
    flipped_image_list.append(np.flip(np.rot90(image), -1))
    flipped_image_list.append(np.flip(np.rot90(image, 3), -1))
    flipped_image_list.append(np.flip(np.flip(np.rot90(image), 0), -1))
    flipped_image_list.append(np.flip(np.flip(np.rot90(image), 1), -1))
images = pd.concat([images, pd.DataFrame({'real_paintings' : pd.Series(flipped_image_list)})], ignore_index = True)
len(images)

4800

In [68]:
Image.fromarray(images['real_paintings'][313]).show()

In [69]:
fake_images_list = []
dir = 'images/photo_jpg'
for image in os.listdir(dir)[0:4800]:
    fake_images_list.append(np.asarray(Image.open(dir +'/' + image)))
images['images_to_transform'] = pd.Series(fake_images_list)
images.head()

,real_paintings,images_to_transform
0,"[[[180, 178, 139], [193, 191, 153], [202, 202,...","[[[113, 110, 101], [113, 110, 101], [112, 112,..."
1,"[[[15, 36, 17], [66, 87, 70], [93, 111, 97], [...","[[[0, 60, 91], [2, 73, 103], [0, 64, 96], [4, ..."
2,"[[[178, 196, 206], [190, 208, 218], [199, 217,...","[[[53, 59, 45], [55, 61, 47], [58, 65, 49], [6..."
3,"[[[160, 164, 191], [163, 169, 195], [164, 170,...","[[[79, 86, 96], [80, 87, 97], [82, 89, 99], [8..."
4,"[[[154, 139, 118], [163, 148, 127], [163, 148,...","[[[131, 173, 195], [130, 172, 194], [129, 171,..."


In [70]:
Image.fromarray(images['images_to_transform'][0]).show()

In [78]:
images.columns

Index(['real_paintings', 'images_to_transform'], dtype='object')

In [98]:
for i in range(0, 5):
    print(images['real_paintings'][i].shape)
    print(images['images_to_transform'][i].shape)

(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)


In [2]:
import tensorflow as tf

In [223]:
class InstanceNormalization(tf.keras.layers.Layer):
    
  def __init__(self, epsilon=1e-5):
    super(InstanceNormalization, self).__init__()
    self.epsilon = epsilon

  def build(self, input_shape):
    self.scale = self.add_weight(
        name='scale',
        shape=input_shape[-1:],
        initializer=tf.random_normal_initializer(1., 0.02),
        trainable=True)

    self.offset = self.add_weight(
        name='offset',
        shape=input_shape[-1:],
        initializer='zeros',
        trainable=True)

  def call(self, x):
    mean, variance = tf.nn.moments(x, axes=[1, 2], keepdims=True)
    inv = tf.math.rsqrt(variance + self.epsilon)
    normalized = (x - mean) * inv
    return self.scale * normalized + self.offset

In [209]:
def downsample(filters, size, norm_type='batchnorm', apply_norm=True):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))

  if apply_norm:
    if norm_type.lower() == 'batchnorm':
      result.add(tf.keras.layers.BatchNormalization())
    elif norm_type.lower() == 'instancenorm':
      result.add(InstanceNormalization())

  result.add(tf.keras.layers.LeakyReLU())

  return result

In [210]:
def upsample(filters, size, norm_type='batchnorm', apply_dropout=False):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
      tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                      padding='same',
                                      kernel_initializer=initializer,
                                      use_bias=False))

  if norm_type.lower() == 'batchnorm':
    result.add(tf.keras.layers.BatchNormalization())
  elif norm_type.lower() == 'instancenorm':
    result.add(InstanceNormalization())

  if apply_dropout:
    result.add(tf.keras.layers.Dropout(0.5))

  result.add(tf.keras.layers.ReLU())

  return result

In [211]:
def generator(output_channels, norm_type='batchnorm'):
  down_stack = [
      downsample(64, 4, norm_type, apply_norm=False),  # (bs, 128, 128, 64)
      downsample(128, 4, norm_type),  # (bs, 64, 64, 128)
      downsample(256, 4, norm_type),  # (bs, 32, 32, 256)
      downsample(512, 4, norm_type),  # (bs, 16, 16, 512)
      downsample(512, 4, norm_type),  # (bs, 8, 8, 512)
      downsample(512, 4, norm_type),  # (bs, 4, 4, 512)
      downsample(512, 4, norm_type),  # (bs, 2, 2, 512)
      downsample(512, 4, norm_type),  # (bs, 1, 1, 512)
  ]

  up_stack = [
      upsample(512, 4, norm_type, apply_dropout=True),  # (bs, 2, 2, 1024)
      upsample(512, 4, norm_type, apply_dropout=True),  # (bs, 4, 4, 1024)
      upsample(512, 4, norm_type, apply_dropout=True),  # (bs, 8, 8, 1024)
      upsample(512, 4, norm_type),  # (bs, 16, 16, 1024)
      upsample(256, 4, norm_type),  # (bs, 32, 32, 512)
      upsample(128, 4, norm_type),  # (bs, 64, 64, 256)
      upsample(64, 4, norm_type),  # (bs, 128, 128, 128)
  ]

  initializer = tf.random_normal_initializer(0., 0.02)
  last = tf.keras.layers.Conv2DTranspose(
      output_channels, 4, strides=2,
      padding='same', kernel_initializer=initializer,
      activation='tanh')  # (bs, 256, 256, 3)

  concat = tf.keras.layers.Concatenate()

  inputs = tf.keras.layers.Input(shape=[None, None, 3])
  x = inputs

  skips = []
  for down in down_stack:
    x = down(x)
    skips.append(x)

  skips = reversed(skips[:-1])

  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = concat([x, skip])

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [212]:
def discriminator(norm_type='batchnorm', target=True):
  initializer = tf.random_normal_initializer(0., 0.02)

  inp = tf.keras.layers.Input(shape=[None, None, 3], name='input_image')
  x = inp

  if target:
    tar = tf.keras.layers.Input(shape=[None, None, 3], name='target_image')
    x = tf.keras.layers.concatenate([inp, tar])  # (bs, 256, 256, channels*2)

  down1 = downsample(64, 4, norm_type, False)(x)  # (bs, 128, 128, 64)
  down2 = downsample(128, 4, norm_type)(down1)  # (bs, 64, 64, 128)
  down3 = downsample(256, 4, norm_type)(down2)  # (bs, 32, 32, 256)

  zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3)  # (bs, 34, 34, 256)
  conv = tf.keras.layers.Conv2D(
      512, 4, strides=1, kernel_initializer=initializer,
      use_bias=False)(zero_pad1)  # (bs, 31, 31, 512)

  if norm_type.lower() == 'batchnorm':
    norm1 = tf.keras.layers.BatchNormalization()(conv)
  elif norm_type.lower() == 'instancenorm':
    norm1 = InstanceNormalization()(conv)

  leaky_relu = tf.keras.layers.LeakyReLU()(norm1)

  zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu)  # (bs, 33, 33, 512)

  last = tf.keras.layers.Conv2D(
      1, 4, strides=1,
      kernel_initializer=initializer)(zero_pad2)  # (bs, 30, 30, 1)

  if target:
    return tf.keras.Model(inputs=[inp, tar], outputs=last)
  else:
    return tf.keras.Model(inputs=inp, outputs=last)

In [214]:
loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [216]:
def discriminator_loss(real, generated):
  return (loss_obj(tf.ones_like(real), real) + loss_obj(tf.zeros_like(generated), generated)) * 0.5

In [217]:
def generator_loss(generated):
  return loss_obj(tf.ones_like(generated), generated)

In [218]:
LAMBDA = 10

In [219]:
def calc_cycle_loss(real_image, cycled_image):
  return LAMBDA * tf.reduce_mean(tf.abs(real_image - cycled_image))

In [220]:
def identity_loss(real_image, same_image):
  return LAMBDA * 0.5 * tf.reduce_mean(tf.abs(real_image - same_image))

In [224]:
OUTPUT_CHANNELS = 3

generator_g = generator(OUTPUT_CHANNELS, norm_type='instancenorm')
generator_f = generator(OUTPUT_CHANNELS, norm_type='instancenorm')

discriminator_x = discriminator(norm_type='instancenorm', target=False)
discriminator_y = discriminator(norm_type='instancenorm', target=False)

In [225]:
generator_g_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_f_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

discriminator_x_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [3]:
@tf.function
def train_step(real_x, real_y):
  # persistent is set to True because the tape is used more than
  # once to calculate the gradients.
  with tf.GradientTape(persistent=True) as tape:
    # Generator G translates X -> Y
    # Generator F translates Y -> X.

    fake_y = generator_g(real_x, training=True)
    cycled_x = generator_f(fake_y, training=True)

    fake_x = generator_f(real_y, training=True)
    cycled_y = generator_g(fake_x, training=True)

    # same_x and same_y are used for identity loss.
    same_x = generator_f(real_x, training=True)
    same_y = generator_g(real_y, training=True)

    disc_real_x = discriminator_x(real_x, training=True)
    disc_real_y = discriminator_y(real_y, training=True)

    disc_fake_x = discriminator_x(fake_x, training=True)
    disc_fake_y = discriminator_y(fake_y, training=True)

    # calculate the loss
    gen_g_loss = generator_loss(disc_fake_y)
    gen_f_loss = generator_loss(disc_fake_x)

    total_cycle_loss = calc_cycle_loss(real_x, cycled_x) + calc_cycle_loss(real_y, cycled_y)

    # Total generator loss = adversarial loss + cycle loss
    total_gen_g_loss = gen_g_loss + total_cycle_loss + identity_loss(real_y, same_y)
    total_gen_f_loss = gen_f_loss + total_cycle_loss + identity_loss(real_x, same_x)

    disc_x_loss = discriminator_loss(disc_real_x, disc_fake_x)
    disc_y_loss = discriminator_loss(disc_real_y, disc_fake_y)

  # Calculate the gradients for generator and discriminator
  generator_g_gradients = tape.gradient(total_gen_g_loss, 
                                        generator_g.trainable_variables)
  generator_f_gradients = tape.gradient(total_gen_f_loss, 
                                        generator_f.trainable_variables)

  discriminator_x_gradients = tape.gradient(disc_x_loss, 
                                            discriminator_x.trainable_variables)
  discriminator_y_gradients = tape.gradient(disc_y_loss, 
                                            discriminator_y.trainable_variables)

  # Apply the gradients to the optimizer
  generator_g_optimizer.apply_gradients(zip(generator_g_gradients, 
                                            generator_g.trainable_variables))

  generator_f_optimizer.apply_gradients(zip(generator_f_gradients, 
                                            generator_f.trainable_variables))

  discriminator_x_optimizer.apply_gradients(zip(discriminator_x_gradients,
                                                discriminator_x.trainable_variables))

  discriminator_y_optimizer.apply_gradients(zip(discriminator_y_gradients,
                                                discriminator_y.trainable_variables))

In [6]:
import time

In [8]:
for epoch in range(10):
  start = time.time()

  n = 0
  for in in range(0, len(images)):
    train_step(images['real_painting'][i], images['images_to_transform'][i])
    if n % 10 == 0:
      print ('.', end='')
    n += 1

  clear_output(wait=True)
  # Using a consistent image (sample_horse) so that the progress of the model
  # is clearly visible.
  generate_images(generator_g, sample_horse)

  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))

  print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                      time.time()-start))

NameError: name 'real_paintings_ds' is not defined